In [16]:
import numpy as np
import os
import re

def load_data():
    # 每封邮件存储成一句话
    files = []
    y = []
    for c in ['spam', 'normal']:
        for file in os.listdir('email/' + c):
            with open('email/' + c + '/' + file) as f:
                lines = []
                for line in f.readlines():
                    line = line.strip()
                    line = [
                        word for word in re.split('\W', line) if len(word) >= 2
                    ]
                    # 每封邮件存储成一句话
                    lines.extend(line)
                files.append(lines)
                y.append(1 if c == 'spam' else 0)
    
    # 所有单词的集合
    words = []
    for line in files:
        words.extend(line)
    # 去重
    words = list(set(words))
    
    # 统计每个单词在每个文件中出现的次数
    x = np.zeros((len(files), len(words)))
    for i in range(len(files)):
        for j in range(len(files[i])):
            if files[i][j] in words:
                x[i, words.index(files[i][j])] += 1
                
    y = np.array(y)
    
    return x, y
    
x, y = load_data()
x.shape, y.shape

((50, 820), (50,))

In [17]:
def train(x, y):
     #首先求总体的违规率,等于 违规次数 / 总次数
    p1 = y.sum() / len(y)
    p0 = 1 - p1
    
    print(p1)

    # 取对数概率
    p1 = np.log(p1)
    p0 = np.log(p0)

    # 垃圾邮件
    x_1 = x[y == 1]
    # 正常邮件
    x_0 = x[y == 0]

    # 统计每个词在垃圾邮件中出现的概率，+1 是为了避免0
    p1_given_word = (x_1.sum(axis=0) + 1) / ((x_1.sum(axis=0) + 1)).sum() 
    #取对数概率
    p1_given_word = np.log(p1_given_word)

    # 统计每个词在正常邮件中出现的概率，+1 是为了避免0
    p0_given_word = (x_0.sum(axis=0) + 1) / ((x_0.sum(axis=0) + 1)).sum()
    p0_given_word = np.log(p0_given_word)

    
    return p1_given_word, p0_given_word, p1, p0
    
p1_given_word, p0_given_word, p1, p0 = train(x, y)
p1_given_word.shape, p0_given_word.shape, p1, p0

0.5


((820,), (820,), -0.6931471805599453, -0.6931471805599453)

In [ ]:
def pred(x):
    p1_given_x = x.dot(p1_given_word) + p1
    p0_given_x = x.dot(p0_given_word) + p0

    return 1 if p1_given_x > p0_given_x else 0


correct = 0
for xi, yi in zip(x, y):
    if pred(xi) == yi:
        correct += 1

correct / len(x)